In [1]:
import pandas as pd
import numpy as np
import re

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)  # me muestre todas las columnas
#pd.set_option('display.max_rows', None)  # me muestre todas las filas

# pa pintar
import pylab as plt   # import matplotlib.pyplot as plt
import seaborn as sns

# para que salga el grafico
%matplotlib inline

In [2]:
df_origin = pd.read_csv(r'..\data\attacks.csv', encoding='latin1')
df = df_origin.copy()

FileNotFoundError: [Errno 2] No such file or directory: '..\\data\\attacks.csv'

In [ ]:
df.head(10)

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
nan_cols = df.isna().sum()

nan_cols[nan_cols>0]

In [ ]:
df.duplicated().any()

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.shape, df_origin.shape

In [ ]:
df.head()

### Número de nulos por cada columna

In [ ]:
nan_cols = df.isna().sum()

nan_cols[nan_cols>0]

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
nan_rows = df.isna().sum(axis=1)

# Filtrar las filas con valores nulos
rows_with_nulls = nan_rows[nan_rows > 0]

rows_with_nulls.tail()

# 1. Limpieza de las columnas (Case Number, Date,	Year, Case Number.1,	Case Number.2,	original order,	Unnamed: 22,	Unnamed: 23	)

### Posición de los valores nulos en 'Date'

In [ ]:
bad_index = df['Date'][df['Date'].isna()].index

bad_index

### Obtener filas por índice o etiqueta de fila

In [ ]:
df.loc[[6302, 6303, 6304, 6305, 6306, 6307, 6308, 6309, 6310, 6311]]

### No hay ninguna, ya que los valores en las columnas 'Case Number' cuando 'Date' es nulo también so nulos

### Relleno los valores nulos de la columna 'Date' 

In [ ]:
df.drop(index=df[df['Date'].isna()].index, inplace=True)
df.shape

In [ ]:
def clean_date(date):
    pattern = r'(\d{2})[/-](\w{3})[/-](\d{4})'  # Patrón regex para identificar la fecha
    match = re.match(pattern, date)  # Buscar el patrón en la fecha

    if match:
        day = match.group(1)  # Extraer el día
        month = match.group(2)  # Extraer el mes
        year = match.group(3)  # Extraer el año

        # Convertir el mes a formato deseado (en este caso, triple abreviatura)
        month_dict = {
            'Jan': 'Jan', 'Feb': 'Feb', 'Mar': 'Mar', 'Apr': 'Apr', 'May': 'May', 'Jun': 'Jun',
            'Jul': 'Jul', 'Aug': 'Aug', 'Sep': 'Sep', 'Oct': 'Oct', 'Nov': 'Nov', 'Dec': 'Dec'
        }

        if month in month_dict:
            month = month_dict[month]

        # Devolver la fecha en el formato deseado
        return f'{day}-{month}-{year}'

    else:
        return None  # Devolver None si no se encuentra el patrón en la fecha

In [ ]:
df['Date'] = df['Date'].apply(clean_date)

df['Date'].head()

In [ ]:
df['Date'].isna().sum()

### Cambio el tipo de dato de la columna 'Date' a datetime

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%Y')

df['Date'].head()

## 1.1 Funcion para limpiar las columnas de 'Case Numbers_' 

In [ ]:
# limpio la columna con una funcion regex



def clean_date_case_number(case_number):
    case_number = str(case_number)
    pattern = r'(\d{4})\.(\d{2})\.(\d{2})'
    match = re.search(pattern, case_number)

    if match:
        year = int(match.group(1))
        month = int(match.group(2))
        day = int(match.group(3))

        if month == 0 or day == 0:
            return np.nan

        try:
            return pd.to_datetime(f'{year}-{month:02d}-{day:02d}')
        except ValueError:
            return np.nan
    else:
        return np.nan

In [ ]:
df['Case Number'] = df['Case Number'].apply(clean_date_case_number)
df['Case Number'].head()

### Paso el tipo de dato de la columna 'Case Number' a datetime

In [ ]:
df['Case Number'] = pd.to_datetime(df['Case Number'], format='%Y-%m-%d')
df['Case Number']

In [ ]:
df['Case Number'].isna().sum()

### Relleno los valores nulos de 'Date' con los de 'Case Number'

In [ ]:
df['Date'] = df['Date'].combine_first(df['Case Number'])

df['Date']

### Valores nulos ahora despues de haber hecho el filtro

In [ ]:
df['Date'].isna().sum()

In [ ]:
df.drop(index=df[df['Date'].isna()].index, inplace=True)
df.shape

### Limpiar valores en 'Case Numbre.1'

In [ ]:
df['Case Number.1'] = df['Case Number.1'].apply(clean_date_case_number)
df['Case Number.1'].head()

### Paso el tipo de dato de la columna 'Case Number.1' a datetime

In [ ]:
df['Case Number.1'] = pd.to_datetime(df['Case Number.1'], format='%Y-%m-%d')
df['Case Number.1']

### Relleno los valores nulos de 'Date' con los de 'Case Number.1'

In [ ]:
df['Date'] = df['Date'].combine_first(df['Case Number.1'])

df['Date']

### Limpiar valores en 'Case Numbre.2'

In [ ]:
df['Case Number.2'] = df['Case Number.2'].apply(clean_date_case_number)
df['Case Number.2'].head()

### Paso el tipo de dato de la columna 'Case Number.2' a datetime

In [ ]:
df['Case Number.2'] = pd.to_datetime(df['Case Number.2'], format='%Y-%m-%d')
df['Case Number.2']

### Relleno los valores nulos de 'Date' con los de 'Case Number.2'


In [ ]:
df['Date'] = df['Date'].combine_first(df['Case Number.2'])

df['Date']

In [ ]:
df['Date'].isna().sum()

### Reinicio el indice con las fechas ( hacer este paso al final, cuando el df esté limpio del todo)

In [ ]:
df_indate = df.set_index('Date')
df_indate.head(20)

# 2. Limpiar Type

In [ ]:
df['Type'].unique()

In [ ]:
df['Type'] = df['Type'].str.replace('Boating', 'Boat')
df['Type'] = df['Type'].str.replace('Boatomg', 'Boat')
df['Type'].unique()

In [ ]:
'''
'Boat': Se refiere a un ataque de tiburón que ocurrió mientras la víctima se encontraba en un bote o embarcación.

'Unprovoked': Representa un ataque de tiburón en el que la víctima fue atacada sin provocación aparente por parte de la persona.

'Invalid': Este valor se utiliza cuando la información disponible no es suficiente para determinar si el incidente 
fue un ataque de tiburón válido o si fue causado por otra causa.

'Provoked': Indica que el ataque ocurrió como respuesta a una provocación o interacción directa con el tiburón por parte 
de la persona, como tocarlo o molestarlo de alguna manera.

'Questionable': Se utiliza cuando la categorización del ataque de tiburón no está clara y se requiere una mayor investigación 
o información para determinar su naturaleza.

'Sea Disaster': Este valor se aplica cuando el incidente no fue un ataque de tiburón directo, sino más bien una situación en 
la que muchas personas estuvieron involucradas y / o personas murieron debido a un desastre marino, como un naufragio, sin 
ser ocasionado específicamente por un tiburón.
'''

In [ ]:
# Cambiar questionable por un nan
df['Type'].value_counts()

In [ ]:
df['Type'] = np.where(df['Type'].isin(['Questionable']), np.nan, df['Type'])
df['Type'].unique()

# 3. Limpiar Country, Location y Area

## 3.1 Country

In [ ]:
df['Country'].unique()

In [ ]:
def clean_strings(x):
    x = str(x) # convertir a string
    pattern= re.findall(r'[a-zA-Z\(\)\-\?]+', x) # buscar patron regex
    if x == 'Unknown':
        return None
    elif pattern:
        return x.lower()

In [ ]:
df['Country'] = df['Country'].apply(clean_strings)
df['Country']

In [ ]:
df['Country'] = df['Country'].str.replace(r'[(\)\-\?]+', '')

In [ ]:
df['Country'].unique()

## 3.2 Area

In [ ]:
df['Area'].unique()

In [ ]:
def clean_location(df, columna):
    df[columna] = df[columna].astype(str)
    df[columna] = df[columna].apply(lambda x: re.sub(r'(\d+|º)', '', x))  # eliminar datos numéricos y símbolo º
    df[columna] = df[columna].apply(lambda x: re.sub(r'\b(west|north|south|east)\b', '', x, flags=re.IGNORECASE))  # eliminar palabras "west", "north", "south" y "east"
    df[columna] = df[columna].apply(lambda x: x.strip())  # eliminar espacios en blanco al inicio o final
    df[columna] = np.where(df[columna].str.isnumeric(), np.nan, df[columna])  # valores numéricos como NaN
    return df

In [ ]:
clean_location(df, 'Area')

In [ ]:
valores_a_buscar = ['N, E', 'N, W', '.N .W', 'S, E']
df.loc[df['Area'].isin(valores_a_buscar), 'Area'] = np.nan

df['Area'].unique()

## 3.3 Location

In [ ]:
df['Location'].unique()

In [ ]:
len(df['Location'].unique())

In [ ]:
df['Location'] = df['Location'].str.replace(r'[(\)\-\?]+', '')

# 4. Limpieza Activity e Injury

## 4.1 Activity

In [ ]:
df['Activity'].unique()

In [ ]:
len(df['Activity'].unique())

In [ ]:
df['Activity'] = df['Activity'].apply(clean_strings)

## 4.2 Injury

In [ ]:
df['Injury'].unique()

In [ ]:
len(df['Injury'].unique())

In [ ]:
df['Activity'] = df['Activity'].apply(clean_strings)

# 5. Limpieza Name

In [ ]:
df['Name'].unique()

In [ ]:
len(df['Name'].unique())

In [ ]:
df['Name'] = np.where(df['Name'].str.lower().isin(['male', 'female']), np.nan, df['Name'])
df.head(20)

# 8. Limpieza Sex y Age

## 8.1 Sex

In [ ]:
df['Sex '].unique()

In [ ]:
def limpiar_genero(df, columna):
    df[columna] = df[columna].astype(str)
    df[columna] = df[columna].apply(lambda x: re.sub(r'[^\w\s]','',x))  # eliminar todos los símbolos
    df[columna] = df[columna].apply(lambda x: re.sub(r'\s+','',x))  # eliminar espacios en blanco
    df[columna] = np.where(df[columna].str.contains('^F|^f|^Fe|^fe', regex=True), 'F',
                           np.where(df[columna].str.contains('^M|^m|^Ma|^ma', regex=True), 'M', np.nan))
    return df
    

In [ ]:
limpiar_genero(df, 'Sex ')

In [ ]:
df['Sex '].unique()

## 8.2 Age

In [ ]:
df['Age'].unique()

In [ ]:
def age_filt(x):
    x=str(x)
    age_pattern = r"\b(\d{2})\b"  # Extracts two-digit numbers
    match = re.search(age_pattern, x)
    
    if match:
        age = int(match.group(1))
        return int(age)
    else:
        return np.nan

In [ ]:
df['Age'] = df['Age'].apply(age_filt)
df['Age']

In [ ]:
df['Age'].unique()

# 9. Limpieza 'Fatal'

In [ ]:
df['Fatal (Y/N)'].unique()

In [ ]:
df['Fatal (Y/N)'].value_counts()

In [ ]:
def clean_fatal(x):
    x=str(x)
    pattern1 = r"[nN]"
    pattern2 = r"[yY]"

    if x == 'UNKNOWN':
        
        return np.nan
    
    elif re.findall(pattern1,x):
        
        return False
    
    elif re.findall(pattern2,x):
        
        return True
        

In [ ]:
df['Fatal (Y/N)'] = df['Fatal (Y/N)'].apply(clean_fatal)
df['Fatal (Y/N)'].unique()

In [ ]:
df['Fatal (Y/N)'].value_counts()

# 10. Limpieza Time

In [ ]:
df['Time'].unique()

In [ ]:
df.head(20)

In [ ]:
def limpiar_hora(hora):
    if isinstance(hora, str):
        hora = hora.strip().lower()
        if 'between' in hora:
            hora = hora.split('between')[-1].strip()
        if 'sometime' in hora:
            hora = hora.split('sometime')[-1].strip()

        # Limpieza del formato hhoo
        if 'oo' in hora:
            hora = hora.replace('oo', '00')
        
        # Limpieza del formato hhmm
        if hora.endswith('h') or hora.endswith('hh'):
            hora = hora[:-1].strip()
            if len(hora) < 4:
                hora = hora.zfill(4)
        
        if hora.startswith('just before') or hora.startswith('just after'):
            hora = hora.split('before')[-1].strip()
            hora = hora.split('after')[-1].strip()
        
        # Limpieza de otros formatos
        formatos_fecha = ['%Hh%M', '%Hh%M:%S', '%H:%M']
        for formato in formatos_fecha:
            try:
                hora_dt = pd.to_datetime(hora, format=formato)
                return hora_dt.time()
            except ValueError:
                continue
    
    return np.nan

In [ ]:
df['Time'] = df['Time'].apply(limpiar_hora)
df['Time'].head(20)

# 11. Limpieza Species

In [ ]:
df['Species '].unique()

In [ ]:
len(df['Species '].unique())

In [ ]:
df['Species '].value_counts()

In [ ]:
df['Species '] = np.where(df['Species '].isin(['Shark involvement prior to death was not confirmed', 'Shark involvement not confirmed', 'Shark involvement prior to death unconfirmed', 'Shark involvement prior to death unconfired', 'Shark involvement not confirmed, injury may be due to a stingray', 'Questionable']), np.nan, df['Species '])
df['Species '].value_counts()

In [ ]:
len(df['Species '].unique())

In [ ]:
df.head(20)

In [ ]:
df.shape

In [ ]:
nan_cols = df.isna().sum()

nan_cols[nan_cols>0]

In [ ]:
df = df.dropna(subset=['Species ']) # Elimina todas las filas que contengan valores nulos en la columna Species del df

In [ ]:
df.shape

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
df.head(20)

In [ ]:
df['Species '].value_counts()

In [ ]:
df['Species '].unique()

In [ ]:
# Creo un dataframe nuevo para analizar las filas que no contienen la palabra 'Shark/shark' dentro de la columna 'Species'

In [ ]:
rows_no_shark = df[~df['Species '].str.contains('shark|Shark', case=False)] # analiza las filas en las que no aparece la palabra Tiburón
rows_no_shark

In [ ]:
rows_no_shark['Species '].unique()

In [ ]:
rows_no_shark['Species '] = np.where(rows_no_shark['Species '].isin(['Unidentified species',
       '15 cm to 20 cm [6" to 8"] bite diameter just below left knee',
       'Species unidentified', '\xa0 ',
       'C. leucas tooth fragment recovered from kayak', 'Unidentified',
       'Questionable incident', 'Unknown', 'Questionable Incident', 'Remains recovered 5 days later', '"Blue whaler" (Galeolamna)', 'Death may have been due to drowning', 'Dooley believed his Injury was caused by stingray (Dasyatidae family)', "[4' to 5']", "3 m [10'] blue whaler", 'Considered a "Doubtful" incident', "4.9 m [16'] whaler"]), np.nan, rows_no_shark['Species '])

In [ ]:
rows_no_shark['Species '].unique()

In [ ]:
rows_no_shark['Species '].value_counts()

In [ ]:
nan_cols = rows_no_shark.isna().sum()

nan_cols[nan_cols>0]

In [ ]:
# ahora aplicamos la limpieza que hemos realizado pero al dataframe con el que estamos trabajando.

In [ ]:
df['Species '] = np.where(df['Species '].isin(['Unidentified species',
       '15 cm to 20 cm [6" to 8"] bite diameter just below left knee',
       'Species unidentified', '\xa0 ',
       'C. leucas tooth fragment recovered from kayak', 'Unidentified',
       'Questionable incident', 'Unknown', 'Questionable Incident', 'Remains recovered 5 days later', '"Blue whaler" (Galeolamna)', 'Death may have been due to drowning', 'Dooley believed his Injury was caused by stingray (Dasyatidae family)', "[4' to 5']", "3 m [10'] blue whaler", 'Considered a "Doubtful" incident', "4.9 m [16'] whaler"]), np.nan, df['Species '])

In [ ]:
nan_cols = df.isna().sum()

nan_cols[nan_cols>0]

In [ ]:
df = df.dropna(subset=['Species '])

In [ ]:
df.shape

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
df.head(20)

# 12. Rellenar nulos del DataFrame

In [ ]:
nan_cols = df.isna().sum()

nan_cols[nan_cols>0]

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
df['Area'] = df['Area'].fillna('unknown')

In [ ]:
df['Location'] = df['Location'].fillna('unknown')

In [ ]:
df['Name'] = df['Name'].fillna('unknown')

In [ ]:
df['Injury'] = df['Injury'].fillna('unknown')

In [ ]:
df['Fatal (Y/N)'] = df['Fatal (Y/N)'].fillna('unknown')

In [ ]:
# Quizá sería convebiente pasar el tipo de dato de esta columna a datetime y rellenar despues los nulos.

df['Time'] = df['Time'].fillna('unknown')

In [ ]:
df['Age'] = df['Age'].fillna(df['Age'].median())

In [ ]:
df['Case Number'] = df['Case Number'].fillna('unknown')

In [ ]:
df['Case Number.1'] = df['Case Number.1'].fillna('unknown')

In [ ]:
df['Case Number.2'] = df['Case Number.2'].fillna('unknown')

In [ ]:
nan_cols = df.isna().sum()

nan_cols[nan_cols>0]

In [ ]:
df['Unnamed: 22'] = df['Unnamed: 22'].fillna('0')

In [ ]:
df['Unnamed: 23'] = df['Unnamed: 23'].fillna('0')

In [ ]:
df['Investigator or Source'] = df['Investigator or Source'].fillna('unknown')

In [ ]:
nan_cols = df.isna().sum()

nan_cols[nan_cols>0]

In [ ]:
# Analizamos que es lo que pasa con el único valor nulo de Year

In [ ]:
bad_index = df['Year'][df['Year'].isna()].index

bad_index

In [ ]:
df.loc[[107]]

In [ ]:
# Como sabemos el año en el que se produjo porque viene especificado en la fecha vamos a modificarlo:
#df.at[i, 'columna'] = nuevo_valor
df.at[107, 'Year'] = 2017

In [ ]:
nan_cols = df.isna().sum()

nan_cols[nan_cols>0]

In [ ]:
df['Case Number'] = df.index
df['Case Number.1'] = df.index
df['Case Number.2'] = df.index
df['original order'] = df.index


In [ ]:
df

In [ ]:
df.duplicated().any()

## Optimización de la memoria cambiando el tipo de dato

In [ ]:
df.info(memory_usage='deep')

In [ ]:
for c in df.select_dtypes(include='object'):
    
    df[c] = df[c].astype('category')   # cambia el tipo a categoria
    
df.info(memory_usage='deep')

In [ ]:
for c in df.select_dtypes('integer'):
    
    df[c] = pd.to_numeric(df[c], downcast='integer')
    
df.info(memory_usage='deep')

In [ ]:
for c in df.select_dtypes('float'):
    
    df[c] = pd.to_numeric(df[c], downcast='float')
    
df.info(memory_usage='deep')

In [ ]:
for c in df.select_dtypes('datetime64[ns]'):
    
    df[c]=df[c].astype('datetime64[s]')
    
df.info(memory_usage='deep')

In [ ]:
df.to_csv(r'..\data\shark_attack_clean.csv', index=False)